In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
#import matplotlib.axes as axes
import re
import os
import calendar

In [2]:
### FLÜSSE UND MESSSTATIONEN AUSLESEN UND SORTIEREN

# einen leeren DataFrame erstellen und die Datei einlesen
df_rv=pd.DataFrame()
dateipfad = 'm_2.txt'
#dateipfad = 'm_all_1.csv'
data=pd.read_csv(dateipfad, sep=';')
data.sort_values(by='grdc_no').reset_index() 
print(data)

# erstellt eine Liste mit den Messstellen-IDs als String
mst_valid_ids = data['grdc_no'].tolist()
mst_valid_lds=[str(i) for i in mst_valid_ids]

# erstellt eine Liste mit den Flussnamen als String
river_ids=data['river'].tolist()
river_ids=[str(i) for i in river_ids]

# erstellt eine Liste mit den Stationsnamen
label_ids=data['station'].tolist()

# Speichern des DataFrames "data" unter neuen Namen (quasi eine Kopie erstellen)
df_ids=data
df_rv_ids=data
#print(df_rv_ids)

# # Spalte 't_start' & 't_end' aus dem DataFrame 'df_rv_ids' in DateTime-Objekte vom Format YYYY-MM-DD umwandeln
# df_rv_ids['t_start'] = pd.to_datetime(df_rv_ids['t_start'])
# #df_rv_ids['t_start'] = df_rv_ids['t_start'].dt.strftime("%Y-%m-%d")
# df_rv_ids['t_end']   = pd.to_datetime(df_rv_ids['t_end'])
# #df_rv_ids['t_end'] = df_rv_ids['t_end'].dt.strftime("%Y-%m-%d")
# #print(df_rv_ids)

# # aus den Start- & Endwerten nur das Jahr extrahieren und in einer neuen Spalte speichern
# df_rv_ids['y_start'] = df_rv_ids['t_start'].dt.year
# df_rv_ids['y_end']   = df_rv_ids['t_end'].dt.year

for mst_id in mst_valid_ids:
    try:
        # aufrufen aller Dateien mit den unterschiedlichen Messstellen-IDs
        path= f'{mst_id}_monthly.txt'
        data = pd.read_csv(path, sep=';')
        # Die Werte der Spalte 'wasserstd_m' extrahieren und in 'df_rv' packen
        df_rv[mst_id] = data['wasserstd_m'] #.tolist()[0:45474]
    except Exception as e:
        print(f'{mst_id} no found!')

print(df_rv)
# print(df_rv_ids)
# print(river_ids)

   grdc_no        river                 station  name_in Pegelonline
0  6335050  RHINE RIVER             DUESSELDORF            Dsseldorf
1  6340180   ELBE RIVER  MAGDEBURG-STROMBRUECKE  magdeburgstrombrcke
           6335050     6340180
0      1880.000000  397.000000
1      2120.000000  388.000000
2      2080.000000  376.000000
3      1960.000000  383.000000
4      1740.000000  389.000000
...            ...         ...
45518  1245.479167  157.041667
45519  1189.291667  153.156250
45520  1155.645833  147.541667
45521  1124.395833  141.895833
45522  1106.071429  140.341463

[45523 rows x 2 columns]


In [3]:
### VARIABLER THRESHOLD

threshold=80
p_threshold=(100-threshold)/100

# leerer DataFrame
df_mt_rv=pd.DataFrame()
plot=False

# für jede Spalte im DataFrame df_rv -> jeder Fluss mit dazugehörigen Werten
for j in df_rv.columns:
    # Datenreihe über die gesamte Länge von df_rv (periods), mit Tagen ('D') als Zeitschritte
    months = pd.date_range(start='1901-01-01', periods=len(df_rv), freq='D')
    # nur die Abflusswerte aus der aktuellen Spalte j aus df_rv als Liste
    data=df_rv[j].tolist()
    #print(len(data))
    # neuer DataFrame, wo wir die Datumsreihe mit den Werten aus df_rv kombinieren
    df = pd.DataFrame({'Date': months, 'Value': data})
    # Datumsspalte als Index setzen
    df['Date']=pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)
    # das Jahr und der Tag des Jahres wird angegeben (aber doppelt, warum?)
    df['D'] = df.index.dayofyear
    df['day_of_year'] = df.index.dayofyear
    #print(df[df['D']==1])
    #print(df.index.dayofyear)

    # Berechnung des 80. Perzentils für jeden Monat

    # alle NaN-Werte durch -999 ersetzen und dann löschen
    df['Value'] = df['Value'].fillna(-999) # diese Zeile hab ich aus '01_fest_threshold_method' übernommen
    df_valid=df[df['Value']!=-999]
    #print(df[df['Value']==-999])
    #print(df_valid.dtypes)

    # Daten in df tageweise gruppieren und dann davon das 80. Quantil bestimmen (quantile(p_threshold))
    mt = df_valid.groupby('D')['Value'].quantile(p_threshold)
    # Berechnung vom Minimum und Maximum in jeder Gruppe 'D' -> wird jeweils in einer Liste gespeichert
    list_b=df_valid.groupby('D')['Value'].min()
    list_a=df_valid.groupby('D')['Value'].max()
    # umwandeln alle Quantil-Werte 'mt' in eine Liste
    mt=mt.tolist()
    #print(len(mt))
    # erstellen von neuem DataFrame mit den Spalten 'day_of_year' und 'threshold'
    threshold_df = pd.DataFrame({'day_of_year': np.arange(1, len(mt)+1), 'threshold': mt})
    # zusammenführen von den beiden DataFrames 'threshold_df' und 'df' anhand der Spalte 'day_of_year'
    df = df.merge(threshold_df, on='day_of_year', how='left')
    #print(df)

    # alle Schwellenwerte mit 31 multiplizieren und die Liste 'mt' auf die ersten 366 Elemente begrenzen
    mt=mt*31
    mt=mt[0:366]
    #print(river_ids[mst_valid_ids.index(j)])

    # im DataFrame 'df_mt_rv' eine neue Spalte mit dem Namen der aktuellen Station [j] erstellen und mit dem Wert des Thresholds füllen
    df_mt_rv[j]=df['threshold']

print(df_mt_rv)


           6335050     6340180
0      1167.022917  311.000000
1      1166.925000  310.600000
2      1198.000000  311.200000
3      1300.000000  300.800000
4      1383.208333  316.000000
...            ...         ...
45518  1298.445833  211.000000
45519  1288.000000  215.016667
45520  1304.000000  215.545833
45521  1292.000000  213.827083
45522  1285.968750  213.991667

[45523 rows x 2 columns]


In [4]:
### DÜRRE BERECHNEN

import datetime

def calculate_drought(q_datas,MT,CDR):   # MT = DataFrame mit den Thresholds, CDR = true/false
    df=pd.DataFrame()
    #Listen, die die Eigenschaften der Dürreereignisse speichern
    ID=[]
    events=[]#0,1,2,3
    duration=[]#unit=month
    mean_deficit=[]#mean_deficit kleiner als 0
    max_deficit=[]
    sum_deficit=[]
    start_time=[]
    end_time=[]
    max_time=[]   # Tag des größten Defizits einer Dürre
    deficit_list=[]
    #duration_list=[]
    deta=[]
    Qs=[]

    # die Daten aus q_datas in eine Liste überführen
    amean=q_datas.tolist()
    #print(amean)
    
    # Datenreihe über die gesamte Länge von 'amean' (periods), mit Tagen ('D') als Zeitschritte
    days = pd.date_range(start='1901-01-01', periods=len(amean), freq='D')
    # neuer DataFrame 'df_data' mit den Spalten 'Datum' und 'Value'
    df_data = pd.DataFrame({'Date': days, 'Value': amean})
    # Datumsspalte als Index in 'df_data' setzen
    df_data['Date']=pd.to_datetime(df_data['Date'])
    df_data.set_index('Date', inplace=True)

    # drought flag
    dflag = False 
    i = 0
    index_a = 0
    # Fehlwerte (-999) durch einen sehr hohen Wert (999999) ersetzen, damit sie nicht als Dürre erkannt werden
    amean=[999999 if int(i) ==-999 else i for i in  amean]
    s_time = 0  # Startzeitpunkt des Dürreereignisses
    d = 0  # Dauer des Dürreereignisses

    # Prüfen von jedem Tag in 'amean'
    for data in amean:
        
        MNQ95 = MT[index_a]  # aus dem DataFrame mit dem Threshold den Wert mit dem index_a raussuchen
        
        if data < MNQ95:
            if dflag==False:   # Beginn einer neuen Dürre
                dflag=True 
                ID.append(i)   # der Dürre eine ID geben
                i=i+1
                deta.append(index_a-(s_time+d))
                s_time=index_a   # setzt die Startzeit
                start_time.append(s_time)
                #print(s_time)
                d=1
                Q=[]   # der Abfluss wird gespeichert
                Q.append(data)
                deficit=[]
                deficit.append(MNQ95-data)
            elif dflag==True: # während der Dürre
                # den aktuellen Abfluss und das Defizit speichern
                deficit.append(MNQ95-data)
                Q.append(data)
                d=d+1
                
        elif data >= MNQ95:
            if dflag==True:   
                # Ende einer Dürre
                duration.append(d)   # Dauer der Dürre wird in der Liste 'duration' gespeichert
                mean_deficit.append(sum(deficit)/len(deficit))   # das durchschnittliche Defizit wird berechnet und wird in der Liste 'mean_deficit' gespeichert
                sum_deficit.append(sum(deficit))   # Gesamtsumme der Defizite dieser Dürre wird in der Liste 'sum_deficit' gespeichert
                max_deficit.append(max(deficit))   # der maximale Wert des Defizits während der Dürre wird bestimmt und zur Liste 'max_deficit' hinzugefügt
                # index_m:
                # max(deficit) sucht den maximalen Wert aus der Liste 'deficit' (die Liste beinhaltet immer nur die Werte der aktuellen Dürre)
                # deficit.index bestimmt den index von diesem maximalen Wert innerhalb der Liste 'deficit'
                # dann wird das zu index_a addiert, aber ich glaube es müsste eigentlich s_time sein
                index_m=s_time+deficit.index(max(deficit)) 
                max_time.append(index_m)   # der berechnete Zeitpunkt von max(deficit) wird zur Liste 'max_time' hinzugefügt
                deficit_list.append(deficit)   # die gesamte Liste mit Defiziten wird zur Liste 'deficit_list' hinzugefügt
                end_time.append(s_time+d)   # Index des Endzeitpunkts wird zur Liste 'end_time' hinzugefügt
                Qs.append(min(Q))   # der minimale Abflusswert wirt zur Liste 'Qs' hinzugefügt
                #maxd=-max(deficit)
                dflag=False

        # für den letzten Tag in unserer Datenreihe, weil der sonst verloren gehen würde wenn er Teil einer Dürre ist
        if data == amean[-1]:
            if dflag==True:
                duration.append(d)
                mean_deficit.append(sum(deficit)/len(deficit))
                sum_deficit.append(sum(deficit))
                max_deficit.append(max(deficit))
                index_m=index_a+deficit.index(max(deficit))
                max_time.append(index_m)
                deficit_list.append(deficit)
                end_time.append(s_time+d)
                Qs.append(min(Q))
                dflag=False

        # falls CDR=true, dflag=true und Monatsende: Dürre wird beendet
        if CDR == True:
            if dflag==True:
                if df_data.index.is_month_end[index_a]:
                    duration.append(d)
                    mean_deficit.append(sum(deficit)/len(deficit))
                    sum_deficit.append(sum(deficit))
                    max_deficit.append(max(deficit))
                    index_m=index_a+deficit.index(max(deficit))
                    max_time.append(index_m)
                    deficit_list.append(deficit)
                    end_time.append(s_time+d)
                    Qs.append(min(Q))
                    dflag=False

        index_a=index_a+1
        
        
    from datetime import date
    from dateutil.relativedelta import relativedelta

    # Umwandlung des Index in ein richtiges Datum ("Tage seit 1901-01-01")
    def m2y(m):
        startdatum = date(1901, 1, 1)  # Reference start date
        neudate = startdatum + relativedelta(days=m)  # Berechnung des neuen Datums durch Hinzufügen von 'm' Tagen zum Startdatum
        years = neudate.year - startdatum.year  # Berechnung der Jahre zwischen dem Startdatum und dem neuen Datum
        months = neudate.month - startdatum.month  # Berechnung der Differenz in Monaten zwischen den beiden Daten
        if months < 0:  # Handle negative months (when crossing a year)
            years -= 1
            months += 12
        years=1901+years   # neues Jahr berechnen
        months=months+1   # Monatszahl um eins erhöhen, um sicherzustellen, dass sie im Bereich von 1 bis 12 bleibt (anstatt von Null bis elf)
        # Die Funktion gibt drei Werte zurück:
            # Anzahl der vergangenen Jahre
            # Anzahl der vergangenen Monate,
            # neu berechnetes Datum (neudate)
        return years, months, neudate  # Return as (years, months)
       
    # Berechnung der Start- & Enddaten der Dürreereignisse
    # für jede Zeitangabe (i) in start_time wird die Funktion m2y aufgerufen
    time_months=[m2y(i)[1] for i in start_time]    # der zweite Wert (Monate) aus dem Ergebnis wird zur Liste time_months hinzugefügt
    time_years=[m2y(i)[0] for i in start_time]    # der erste Wert (Jahre) geht an 'time_years'
    time_date=[m2y(i)[2] for i in start_time]   # ...
    end_date=[m2y(i)[2] for i in end_time]

    # für jede Zeitangabe (i) in 'max_time' wird erneut die Funktion aufgerufen
    time_date_max =[m2y(i)[2] for i in max_time]
    time_month_max =[m2y(i)[1] for i in max_time]

    # alle Werte aus den Listen in den DataFrame 'df' implementieren
    df['ID']=ID
    df['duration']=duration
    df['time_deta']=deta   #[i if i<=365 else 0 deta]
    df['mean_deficit']=mean_deficit
    df['max_deficit']=max_deficit
    df['sum_deficit']=sum_deficit
    df['list_deficit']=deficit_list
    df['Qmin']=Qs
    df['start_date']=time_date
    df['start_year']=time_years
    df['start_month']=time_months
    df['max_date']=time_date_max
    df['max_month']=time_month_max
    df['end_date']=end_date
    df['start_year']=time_years
    df['start_month']=time_months
                
    return df

    
#for i in range(len(ID)):
    #print('ID=',ID[i])
    #print('event type=',events[i])
    #print('duration=',duration[i])
    #print('mean deficit',mean_deficit[i])
    #print('max deficit',max_deficit[i])
    #print('start time=',m2y(start_time[i]))
    #print('\n')

In [5]:
### ERGEBNIS AUSGEBEN LASSEN

df_result=pd.DataFrame()

# die beiden Listen df_rv.columns und river_ids so kombinieren, dass für jede Iteration i den aktuellen Spaltennamen und j die entsprechende Fluss-ID erhält
for i,j in zip(df_rv.columns,river_ids):
    # ersetzen der NaN-Werte in df_rv durch -999 als int
    df_rv[i] = df_rv[i].fillna(-999).astype(int)
    # aufrufen der vorher definierten Funktion "calculate_drought" für die aktuelle Spalte i
    # df_mt_rv[i].tolist(): konvertiert die aktuelle Spalte von 'df_mt_rv' (Thresholds) in eine Liste
    # True: bezieht sich auf CDR - eine Dürre wird am Monatsende beendet
    df_ssi_c1=calculate_drought(df_rv[i],df_mt_rv[i].tolist(),True)
    #df_ssi_c1.to_csv(f'rv_mst_result/VTM_{threshold}/drought_result_test.csv',sep=';')

    # erstellen von neuen Spalten im DataFrame 'df_ssi_c1' -> für jede Zeile Name der Station / Fluss
    df_ssi_c1['Station']=[i]*len(df_ssi_c1)
    df_ssi_c1['River']=[j]*len(df_ssi_c1)
    # den DataFrame df_ssi_c1 an das Ende des bereits existierenden DataFrames df_result anhängen
    df_result = pd.concat([df_result, df_ssi_c1], ignore_index=True)

# Dürreereignisse mit einer Dauer von mindestens 5 Tagen
#df_result=df_result[df_result['duration']>=5]

# Dürreereignisse mit einem Gesamtdefizit von mindestens 5
#df_result=df_result[df_result['sum_deficit']>=5]

# eine Kopie von 'df_result' speichern
df_result0=df_result
# die Datei im zugehörigen Ordner speichern
df_result.to_csv(f'rv_mst_result/VTM_{threshold}/drought_result_all.csv',sep=';')

# Entfernen von doppelten Fluss-IDs
river_id=list(set(river_ids))

#print(df_result)
#print(df_result0)

In [6]:
### STATISTIK

# Definition der Variablen a und b durch die Spalten eines DataFrames
a='duration'
b='max_deficit'
# leeren DataFrame erstellen
df_result_s=pd.DataFrame()

# Schleife über die Spaltennamen [i] in 'df_rv' (IDs der Messstationen) und die Fluss-IDs [j]
for i,j in zip(df_rv.columns,river_ids):
    # 'df_test' enthält alle Zeilen aus 'df_result' die den aktuellen Stationsnamen [i] enthalten
    df_test=df_result[df_result['Station']==i]
    x_data1=df_test[f'{a}']
    y_data1=df_test[f'{b}']
    df_s=pd.DataFrame()
    df_s['Station']=[i]
    df_s[f'number_of_events']=[len(x_data1)]
    df_s[f'mean_max_deficit']=[y_data1.mean()]
    df_s[f'mean_duration']=[x_data1.mean()]
    df_s[f'max_max_deficit']=[y_data1.max()]
    df_s[f'max_duration']=[x_data1.max()]
    for season,i in zip(['winter','spring','summer','autumn'],[[11, 12, 1],[2,3, 4],[5, 6, 7],[8, 9, 10]]): 
        df_result1=df_test[df_test['start_month'].isin(i)]
        x_data1=df_result1[f'{a}']
        y_data1=df_result1[f'{b}']
        df_s[f'{season}_number_of_events']=[len(x_data1)]
        df_s[f'{season}_mean_max_deficit']=[y_data1.mean()]
        df_s[f'{season}_mean_duration']=[x_data1.mean()]
        df_s[f'{season}_max_max_deficit']=[y_data1.max()]
        df_s[f'{season}_max_duration']=[x_data1.max()]
    df_result_s = pd.concat([df_result_s, df_s], ignore_index=True)

# das Ergebnis als CSV speichern - ich habe die Direction in den Ordner VTM_threshold verschoben
df_result_s.to_csv(f'rv_mst_result/VTM_{threshold}/drought_result_statistic.csv',sep=';')

In [7]:
### ERGEBNISSE NACH SAISONEN

df_result0=pd.read_csv(f'rv_mst_result/VTM_{threshold}/drought_result_all.csv',sep=';')
statistic=[]

for k,j,start_y,end_y in zip(df_rv.columns,river_ids,df_rv_ids['y_start'].tolist(),df_rv_ids['y_end'].tolist()):
    df_result_all=pd.DataFrame()
    if int(start_y)<1901:
        start_y='1901'
    years=[i+int(start_y) for i in range(int(end_y)-int(start_y)+1)]
    events=[]
    df_result=df_result0[df_result0['Station']==k]
    
    for y in years:
        df_result_s=pd.DataFrame()
        df_year_1=pd.DataFrame()
        df_year_2=pd.DataFrame()
        #df_year=df_result0[df_result0['start_year']==y]
        df_result_s['year']=[y]
        for season,i in zip(['winter','spring','summer','autumn'],[[11, 12, 1],[2,3, 4],[5, 6, 7],[8, 9, 10]]):
                df_year_1=df_result[df_result['start_year']==y]
                
                if season == 'winter':
                    df_1=df_year_1[df_year_1['start_month'].isin([1])]
                    if y>years[0]:
                        df_year_2=df_result[df_result['start_year']==y-1]
                        df_2=df_year_2[df_year_2['start_month'].isin([11,12])]
                        df_season=pd.concat([df_1, df_2], ignore_index=True)
                    else:
                        df_season=df_1
                else:
                    df_season=df_year_1[df_year_1['start_month'].isin(i)]
                if len(df_season) ==0:
                    df_result_s[f'{season}_cumulative_events']=[0]
                    df_result_s[f'{season}_cumulative_duration']=[0]
                    df_result_s[f'{season}_mean_duration']=[0]
                    df_result_s[f'{season}_cumulative_deficit']=[0]
                    df_result_s[f'{season}_max_deficit']=[0]
                else:
                    df_result_s[f'{season}_cumulative_events']=[1]
                    df_result_s[f'{season}_cumulative_duration']=[df_season['duration'].sum()]
                    df_result_s[f'{season}_mean_duration']=[df_season['duration'].mean()]
                    df_result_s[f'{season}_cumulative_deficit']=[df_season['sum_deficit'].sum()]
                    df_result_s[f'{season}_max_deficit']=[df_season['max_deficit'].max()]

        df_result_all = pd.concat([df_result_all, df_result_s], ignore_index=True)
    #statistic.append({'Station':k})
    
    statistic.append({
        'Station':k,
        'River':j,
        
        f'mean winter_cumulative_events':df_result_all[f'winter_cumulative_events'].mean(),
        f'mean spring_cumulative_events':df_result_all[f'spring_cumulative_events'].mean(),
        f'mean summer_cumulative_events':df_result_all[f'summer_cumulative_events'].mean(),
        f'mean autumn_cumulative_events':df_result_all[f'autumn_cumulative_events'].mean(),
        
        f'mean winter_cumulative_duration': df_result_all[f'winter_cumulative_duration'].mean(),
        f'mean spring_cumulative_duration': df_result_all[f'spring_cumulative_duration'].mean(),
        f'mean summer_cumulative_duration': df_result_all[f'summer_cumulative_duration'].mean(),
        f'mean autumn_cumulative_duration': df_result_all[f'autumn_cumulative_duration'].mean(),
        
        f'mean winter_max_deficit': df_result_all[f'winter_max_deficit'].mean(), 
        f'mean spring_max_deficit': df_result_all[f'spring_max_deficit'].mean(),       
        f'mean summer_max_deficit': df_result_all[f'summer_max_deficit'].mean(),
        f'mean autumn_max_deficit': df_result_all[f'autumn_max_deficit'].mean(),

        f'max winter_max_deficit': df_result_all[f'winter_max_deficit'].max(), 
        f'max spring_max_deficit': df_result_all[f'spring_max_deficit'].max(),       
        f'max summer_max_deficit': df_result_all[f'summer_max_deficit'].max(),
        f'max autumn_max_deficit': df_result_all[f'autumn_max_deficit'].max(),

        f'max winter_cumulative_duration': df_result_all[f'winter_cumulative_duration'].max(),
        f'max spring_cumulative_duration': df_result_all[f'spring_cumulative_duration'].max(),
        f'max summer_cumulative_duration': df_result_all[f'summer_cumulative_duration'].max(),
        f'max autumn_cumulative_duration': df_result_all[f'autumn_cumulative_duration'].max(),
    })
    
    df_result_all = df_result_all.sort_values(by="year")
    df_result_all.to_csv(f'rv_mst_result/VTM_{threshold}/onset_seasonal_drought_result/{j}_{k}.csv',sep=';')
statistic=pd.DataFrame(statistic)
statistic.to_csv(f'onset_seasonal_drought_statistic.csv',sep=';')
#print(statistic)

KeyError: 'y_start'